- [1] 모듈 로딩 <hr>

In [2]:
### ===> Module Import
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets

In [3]:
### ===> 딥러닝 모델을 설계할 때 활용하는 장비 확인
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
    
print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)

Using PyTorch version: 2.2.1  Device: cpu


- [2] 데이터 준비 <hr>

In [4]:
### ===>  데이터 추출 관련 변수
BATCH_SIZE = 32
EPOCHS = 10

In [5]:
### ===> CIFAR10 데이터 다운로드 (Train set, Test set 분리하기)
train_dataset = datasets.CIFAR10(root = "../data/CIFAR_10",
                                  train = True,  # 학습용
                                  download = True,
                                  transform = transforms.ToTensor())

test_dataset = datasets.CIFAR10(root = "../data/CIFAR_10",
                                train = False,  # 테스트용
                                transform = transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                            batch_size = BATCH_SIZE,
                                            shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = BATCH_SIZE,
                                          shuffle = False)

100.0%


Extracting ../data/CIFAR_10\cifar-10-python.tar.gz to ../data/CIFAR_10


In [15]:
### ===>  데이터 확인하기 (1)
# X_train: torch.Size([32, 3, 32, 32]) [배치사이즈, 채널(컬러 : 3), 높이, 너비]
# y_train: torch.Size([32]) [배치사이즈] 즉, 32개 이미지에 대한 라벨
for (X_train, y_train) in train_loader:
    print('X_train:', X_train.size(), 'type:', X_train.type()) 
    print('y_train:', y_train.size(), 'type:', y_train.type())
    break

X_train: torch.Size([32, 3, 32, 32]) type: torch.FloatTensor
y_train: torch.Size([32]) type: torch.LongTensor


In [16]:
### ===>  데이터 확인하기 (2) 
pltsize = 1
plt.figure(figsize=(10 * pltsize, pltsize))

for i in range(10):
    plt.subplot(1, 10, i + 1)
    plt.axis('off')
    plt.imshow(np.transpose(X_train[i], (1, 2, 0)))
    plt.title('Class: ' + str(y_train[i].item()))

- [3] 모델 설계 <hr>

In [ ]:
from torchviz import make_dot

x=torch.randn(1,3,32,32).to(DEVICE)
y=model(x)

make_dot(y.mean(),params=dict(model.named_parameters()))

In [17]:
### ===>  Convolutional Neural Network (CNN) 모델 설계하기
# X_train: torch.Size([32, 3, 32, 32]) [배치사이즈, 채널(컬러 : 3), 높이, 너비]
# y_train: torch.Size([32]) [배치사이즈] 즉, 32개 이미지에 대한 라벨
# 채널 한개당 커널 한개 / 커널은 퍼셉트론과 동일하다고 생각하면 쉬움
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()       # in_channels : 채널수, out_channels : 커널수
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 8, kernel_size = 3, padding = 1) 
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = 3, padding = 1) # padding = 1: same padding
        self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)  # 사이즈만 줄임 그 외 하는거 없음 - 반으로 줄임 # stride = 2 : 두칸씩 움직여라 / 기본 1 
        self.fc1 = nn.Linear(8 * 8 * 16, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        
        x = x.view(-1, 8 * 8 * 16)  # 이미지 수, 이미지 데이터
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)

        return x

- [4] 학습 준비 <hr>

In [18]:
### ===> Optimizer, Objective Function 설정
model = CNN().to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

criterion = nn.CrossEntropyLoss()

print(model)

CNN(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=10, bias=True)
)


In [20]:
### ===> CNN 모델 학습을 진행하며 학습 데이터에 대한 모델 성능을 확인하는 함수 정의
def train(epoch, model, train_loader, optimizer, log_interval):
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print(f"Train Epoch: {epoch} [{batch_idx}]\tTrain Loss: {loss.item():.6f}")

In [21]:
### ===> 학습되는 과정 속에서 검증 데이터에 대한 모델 성능을 확인하는 함수 정의
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

- [5] 학습진행 <hr>

In [24]:
### ===> CNN 학습 실행하며 Train, Test set의 Loss 및 Test set Accuracy 확인
for epoch in range(1, EPOCHS + 1):
    
    train(epoch, model, train_loader, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print(f"\n[EPOCH: {epoch}], \tTest Loss: {test_loss:.4f}, \tTest Accuracy: {test_accuracy:.2f} % \n")

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_101672\2383047395.py:26: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.log_softmax(x)


Train Epoch: 1 [0]	Train Loss: 2.288496
Train Epoch: 1 [200]	Train Loss: 1.789126
Train Epoch: 1 [400]	Train Loss: 1.680451
Train Epoch: 1 [600]	Train Loss: 1.524914
Train Epoch: 1 [800]	Train Loss: 1.652172
Train Epoch: 1 [1000]	Train Loss: 1.732938
Train Epoch: 1 [1200]	Train Loss: 1.442307
Train Epoch: 1 [1400]	Train Loss: 1.466804


KeyboardInterrupt: 